## MLFlow Tracking Server

In order to start the MLflow tracking server,  go to the folder via cmd or powershell, and type:  mlflow ui

In [1]:
import mlflow



In [3]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment('Check Connection 1')


2024/10/22 19:12:21 INFO mlflow.tracking.fluent: Experiment with name 'Check Connection 1' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/258775168737052725', creation_time=1729620741426, experiment_id='258775168737052725', last_update_time=1729620741426, lifecycle_stage='active', name='Check Connection 1', tags={}>

In [4]:
for i in range(10):
    with mlflow.start_run():
        mlflow.log_metric('Test', i)
        mlflow.log_metric('Jmario', i+1)

2024/10/22 19:12:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run clumsy-sponge-760 at: http://localhost:5000/#/experiments/258775168737052725/runs/73d1b98a92cc4758821e7e5cb17a98cc.
2024/10/22 19:12:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/258775168737052725.
2024/10/22 19:12:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run kindly-mole-440 at: http://localhost:5000/#/experiments/258775168737052725/runs/e1e56315bc934e2fa1e11d25a4e920d2.
2024/10/22 19:12:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/258775168737052725.
2024/10/22 19:12:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run likeable-cub-249 at: http://localhost:5000/#/experiments/258775168737052725/runs/2850ce79a1af4c82bfd5b67ab5c15ff9.
2024/10/22 19:12:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5000/#/experiments/258775